In [1]:
import pandas as pd

from flashtext import KeywordProcessor

import re

import os

import warnings

warnings.filterwarnings("ignore")

In [2]:
account_data_path = r"C:\Users\rohit\settleking_test\data\processed\account_activity.csv"

merchant_data_path = r"C:\Users\rohit\settleking_test\data\processed\merch.csv"

In [3]:
account_df = pd.read_csv(account_data_path, parse_dates = ["Date"])

merch_df = pd.read_csv(merchant_data_path)

In [4]:
account_df["Cheque_Number"] = account_df["Cheque_Number"].astype("Int64")

In [5]:
keyword_processor = KeywordProcessor(case_sensitive = False)

for _,row in merch_df.iterrows():
    keyword = row["alias"]
    category = row["category"]
    keyword_processor.add_keyword(keyword,category)

In [6]:
def categorize(text):

    matches = keyword_processor.extract_keywords(text)

    if matches:

        matches = list(dict.fromkeys(matches))

        matches += [None] * (3-len(matches))

        return matches[:3]
    
    else:
    
        return ["Miscellaneous","None","None"]

In [7]:
priority = [
    "Online Purchase",
    "Entertainment",
    "Utility Payment",
    "Direct Deposit",
    "Card Payment",
    "Travel",
    "Investments/Finance",
    "Utility/Insurance",
    "International Transfer",
    "Food",
    "SettleKing",
    "9LLabs",
    "Rewards",
    "P2P Payments",
    "Processing Fee",
    "Fees",
    "Toll",
    "Tax Payment",
    "B2B Payment",
    "Bank Transfer"
]

In [8]:
def categorize_multi(text):

    match = keyword_processor.extract_keywords(text)

    if match:

        match = list(dict.fromkeys(match))
    
        match_sorted = sorted(

        match,
        key=lambda x: priority.index(x) if x in priority else len(priority)
        )
    
        match_sorted += ["None"] * (3 - len(match_sorted))
    
        return match_sorted[:3]
    
    else:
        
        return ["Miscellaneous","None","None"]

In [9]:
import random

test_range = [random.randint(0,15) for _ in range(10)]

test_desc = [account_df["Cleaned_Desc"][number] for number in test_range]

In [10]:
test_desc

['online transfer from 9llabs llc business checking ref on',
 'online transfer from 9llabs llc business checking ref on',
 'online transfer to 9llabs llc ref business checking loan',
 'state farm ro sfpp s sv',
 'cybrid 25d568e0',
 'bell ch academy epay plan c sv',
 'online transfer to 9llabs llc ref business checking loan',
 'state farm ro sfpp s sv',
 'online transfer from 9llabs llc business checking ref on',
 'business to business ach wsdot good to go good to go a kalvi consulting servi']

In [11]:
for i in range(0,5):
    
    print(f"Description : {test_desc[i]}\n") 

    print(f"Category : {categorize_multi(test_desc[i])}\n")

    print("--------------------------------------------\n")

Description : online transfer from 9llabs llc business checking ref on

Category : ['9LLabs', 'Bank Transfer', 'None']

--------------------------------------------

Description : online transfer from 9llabs llc business checking ref on

Category : ['9LLabs', 'Bank Transfer', 'None']

--------------------------------------------

Description : online transfer to 9llabs llc ref business checking loan

Category : ['9LLabs', 'Bank Transfer', 'None']

--------------------------------------------

Description : state farm ro sfpp s sv

Category : ['Insurance', 'None', 'None']

--------------------------------------------

Description : cybrid 25d568e0

Category : ['International Transfer', 'None', 'None']

--------------------------------------------



In [12]:
account_df[["Category_1","Category_2","Category_3"]] = account_df["Cleaned_Desc"].apply(categorize_multi).apply(pd.Series)

In [13]:
account_df["Category_1"].unique()

array(['Insurance', 'Card Payment', '9LLabs', 'Bank Transfer',
       'SettleKing', 'Toll', 'Processing Fee', 'Fees', 'Rewards',
       'International Transfer', 'P2P Payments', 'Utility Payment',
       'Investments/Finance', 'Tax Payment', 'Miscellaneous',
       'B2B Payment'], dtype=object)

In [14]:
account_df.head(n = 5)

,Date,Amount,abs_Amount,Cheque_Number,Transaction_Type,Cleaned_Desc,Description,Day,Month,Year,Category_1,Category_2,Category_3
0,2024-07-24,-217.67,217.67,<NA>,0,state farm ro sfpp s sv,STATE FARM RO 27 SFPP 15 S 1074398715 SV,24,July,2024,Insurance,None,None
1,2024-07-24,-7500.00,7500.00,<NA>,0,online transfer ref to visa signature card on,ONLINE TRANSFER REF #IB0NYCH2M4 TO VISA SIGNAT...,24,July,2024,Card Payment,Bank Transfer,None
2,2024-07-24,-3875.00,3875.00,<NA>,0,online transfer to 9llabs llc ref business che...,ONLINE TRANSFER TO 9LLABS LLC REF #IB0NYCGZRL ...,24,July,2024,9LLabs,Bank Transfer,None
3,2024-07-18,-330.00,330.00,127,0,deposited or cashed check,DEPOSITED OR CASHED CHECK # 127,18,July,2024,Bank Transfer,None,None
4,2024-07-08,-9975.00,9975.00,<NA>,0,online transfer to 9llabs llc ref business che...,ONLINE TRANSFER TO 9LLABS LLC REF #IB0NSKMG2J ...,8,July,2024,9LLabs,Bank Transfer,None


In [15]:
account_df[account_df["Category_1"] == "Toll"]

,Date,Amount,abs_Amount,Cheque_Number,Transaction_Type,Cleaned_Desc,Description,Day,Month,Year,Category_1,Category_2,Category_3
6,2024-07-05,-1.25,1.25,<NA>,0,business to business ach wsdot good to go good...,BUSINESS TO BUSINESS ACH WSDOT GOOD TO GO GOOD...,5,July,2024,Toll,B2B Payment,Bank Transfer
42,2024-05-20,-1.25,1.25,<NA>,0,business to business ach wsdot good to go good...,BUSINESS TO BUSINESS ACH WSDOT GOOD TO GO GOOD...,20,May,2024,Toll,B2B Payment,Bank Transfer


In [16]:
account_df.groupby(["Category_1","Category_2"]).count()

Date  Amount  abs_Amount  Cheque_Number  \
Category_1             Category_2                                               
9LLabs                 B2B Payment       1       1           1              0   
                       Bank Transfer     7       7           7              0   
B2B Payment            Bank Transfer     2       2           2              0   
Bank Transfer          None             16      16          16              9   
Card Payment           Bank Transfer     7       7           7              0   
Fees                   Bank Transfer     1       1           1              0   
                       None              8       8           8              0   
Insurance              None              7       7           7              0   
International Transfer None              7       7           7              0   
Investments/Finance    None              4       4           4              0   
Miscellaneous          None              4       4           4              0   
P2P Payments           None              7       7           7              0   
Processing Fee         None              1       1           1              0   
Rewards                None              4       4           4              0   
SettleKing             Bank Transfer     1       1           1              0   
                       None             16      16          16              0   
Tax Payment            None              1       1           1              0   
Toll                   B2B Payment       2       2           2              0   
Utility Payment        None              3       3           3              0   

                                      Transaction_Type  Cleaned_Desc  \
Category_1             Category_2                                      
9LLabs                 B2B Payment                   1             1   
                       Bank Transfer                 7             7   
B2B Payment            Bank Transfer                 2             2   
Bank Transfer          None                         16            16   
Card Payment           Bank Transfer                 7             7   
Fees                   Bank Transfer                 1             1   
                       None                          8             8   
Insurance              None                          7             7   
International Transfer None                          7             7   
Investments/Finance    None                          4             4   
Miscellaneous          None                          4             4   
P2P Payments           None                          7             7   
Processing Fee         None                          1             1   
Rewards                None                          4             4   
SettleKing             Bank Transfer                 1             1   
                       None                         16            16   
Tax Payment            None                          1             1   
Toll                   B2B Payment                   2             2   
Utility Payment        None                          3             3   

                                      Description  Day  Month  Year  \
Category_1             Category_2                                     
9LLabs                 B2B Payment              1    1      1     1   
                       Bank Transfer            7    7      7     7   
B2B Payment            Bank Transfer            2    2      2     2   
Bank Transfer          None                    16   16     16    16   
Card Payment           Bank Transfer            7    7      7     7   
Fees                   Bank Transfer            1    1      1     1   
                       None                     8    8      8     8   
Insurance              None                     7    7      7     7   
International Transfer None                     7    7      7     7   
Investments/Finance    None               

In [17]:
account_df["round_Amount"] = account_df["Amount"].round(2)

In [18]:
account_sorted_df = account_df.sort_values(["Category_1","Date","round_Amount"])

In [19]:
account_sorted_df

,Date,Amount,abs_Amount,Cheque_Number,Transaction_Type,Cleaned_Desc,Description,Day,Month,Year,Category_1,Category_2,Category_3,round_Amount
97,2024-01-24,5000.00,5000.00,<NA>,1,9llabs llc direct dep 93153174682489f sv,9LLABS LLC DIRECT DEP 240124 93153174682489F SV,24,January,2024,9LLabs,Bank Transfer,None,5000.00
91,2024-02-01,20000.00,20000.00,<NA>,1,online transfer from 9llabs llc business check...,ONLINE TRANSFER FROM 9LLABS LLC BUSINESS CHECK...,1,February,2024,9LLabs,Bank Transfer,None,20000.00
76,2024-02-27,-1457.25,1457.25,<NA>,0,business to business ach wa state dol wa st dm...,BUSINESS TO BUSINESS ACH WA STATE DOL WA ST DM...,27,February,2024,9LLabs,B2B Payment,Bank Transfer,-1457.25
48,2024-05-01,25000.00,25000.00,<NA>,1,online transfer from 9llabs llc ref business c...,ONLINE TRANSFER FROM 9LLABS LLC REF #IB0N2W75G...,1,May,2024,9LLabs,Bank Transfer,None,25000.00
46,2024-05-08,-20520.00,20520.00,<NA>,0,online transfer to 9llabs llc ref business che...,ONLINE TRANSFER TO 9LLABS LLC REF #IB0N5DX48M ...,8,May,2024,9LLabs,Bank Transfer,None,-20520.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2024-05-20,-1.25,1.25,<NA>,0,business to business ach wsdot good to go good...,BUSINESS TO BUSINESS ACH WSDOT GOOD TO GO GOOD...,20,May,2024,Toll,B2B Payment,Bank Transfer,-1.25
6,2024-07-05,-1.25,1.25,<NA>,0,business to business ach wsdot good to go good...,BUSINESS TO BUSINESS ACH WSDOT GOOD TO GO GOOD...,5,July,2024,Toll,B2B Payment,Bank Transfer,-1.25
90,2024-02-01,-578.62,578.62,<NA>,0,city of bellevu util bill sv,CITY OF BELLEVU UTIL BILL 240201 18688005 SV,1,February,2024,Utility Payment,None,None,-578.62
57,2024-04-04,-616.97,616.97,<NA>,0,city of bellevu util bill sv,CITY OF BELLEVU UTIL BILL 240404 18688005 SV,4,April,2024,Utility Payment,None,None,-616.97


In [20]:
account_sorted_df["Previous_Day"] = account_sorted_df.groupby("Category_1")["Date"].shift(1)

In [21]:
account_sorted_df["Gap_Days"] = (account_sorted_df["Date"] - account_sorted_df["Previous_Day"]).dt.days

In [22]:
account_sorted_df["Rec_30"] = account_sorted_df["Gap_Days"].between(20,40)

account_sorted_df["Rec_60"] = account_sorted_df["Gap_Days"].between(50,70)

account_sorted_df["Rec_90"] = account_sorted_df["Gap_Days"].between(85,95)

In [23]:
account_sorted_df["RecurringStreak_30"] = account_sorted_df.groupby("Category_1")["Rec_30"].cumsum()

account_sorted_df["RecurringStreak_60"] = account_sorted_df.groupby("Category_1")["Rec_60"].cumsum()

account_sorted_df["RecurringStreak_90"] = account_sorted_df.groupby("Category_1")["Rec_90"].cumsum()

In [24]:
account_sorted_df["Is_Recurring_30"] = account_sorted_df["RecurringStreak_30"].map(lambda x: 1 if x >= 2 else 0)

account_sorted_df["Is_Recurring_60"] = account_sorted_df["RecurringStreak_60"].map(lambda x: 1 if x >= 2 else 0)

account_sorted_df["Is_Recurring_90"] = account_sorted_df["RecurringStreak_90"].map(lambda x: 1 if x >= 2 else 0)

In [25]:
account_sorted_df

,Date,Amount,abs_Amount,Cheque_Number,Transaction_Type,Cleaned_Desc,Description,Day,Month,Year,...,Gap_Days,Rec_30,Rec_60,Rec_90,RecurringStreak_30,RecurringStreak_60,RecurringStreak_90,Is_Recurring_30,Is_Recurring_60,Is_Recurring_90
97,2024-01-24,5000.00,5000.00,<NA>,1,9llabs llc direct dep 93153174682489f sv,9LLABS LLC DIRECT DEP 240124 93153174682489F SV,24,January,2024,...,NaN,False,False,False,0,0,0,0,0,0
91,2024-02-01,20000.00,20000.00,<NA>,1,online transfer from 9llabs llc business check...,ONLINE TRANSFER FROM 9LLABS LLC BUSINESS CHECK...,1,February,2024,...,8.0,False,False,False,0,0,0,0,0,0
76,2024-02-27,-1457.25,1457.25,<NA>,0,business to business ach wa state dol wa st dm...,BUSINESS TO BUSINESS ACH WA STATE DOL WA ST DM...,27,February,2024,...,26.0,True,False,False,1,0,0,0,0,0
48,2024-05-01,25000.00,25000.00,<NA>,1,online transfer from 9llabs llc ref business c...,ONLINE TRANSFER FROM 9LLABS LLC REF #IB0N2W75G...,1,May,2024,...,64.0,False,True,False,1,1,0,0,0,0
46,2024-05-08,-20520.00,20520.00,<NA>,0,online transfer to 9llabs llc ref business che...,ONLINE TRANSFER TO 9LLABS LLC REF #IB0N5DX48M ...,8,May,2024,...,7.0,False,False,False,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2024-05-20,-1.25,1.25,<NA>,0,business to business ach wsdot good to go good...,BUSINESS TO BUSINESS ACH WSDOT GOOD TO GO GOOD...,20,May,2024,...,NaN,False,False,False,0,0,0,0,0,0
6,2024-07-05,-1.25,1.25,<NA>,0,business to business ach wsdot good to go good...,BUSINESS TO BUSINESS ACH WSDOT GOOD TO GO GOOD...,5,July,2024,...,46.0,False,False,False,0,0,0,0,0,0
90,2024-02-01,-578.62,578.62,<NA>,0,city of bellevu util bill sv,CITY OF BELLEVU UTIL BILL 240201 18688005 SV,1,February,2024,...,NaN,False,False,False,0,0,0,0,0,0
57,2024-04-04,-616.97,616.97,<NA>,0,city of bellevu util bill sv,CITY OF BELLEVU UTIL BILL 240404 18688005 SV,4,April,2024,...,63.0,False,True,False,0,1,0,0,0,0


In [26]:
account_sorted_df.groupby("Is_Recurring_30")["Is_Recurring_30"].count()

Is_Recurring_30
0    78
1    21
Name: Is_Recurring_30, dtype: int64

In [27]:
account_sorted_df.groupby("Is_Recurring_60")["Is_Recurring_60"].count()

Is_Recurring_60
0    94
1     5
Name: Is_Recurring_60, dtype: int64

In [28]:
account_sorted_df[account_sorted_df["Is_Recurring_60"] == 1]

,Date,Amount,abs_Amount,Cheque_Number,Transaction_Type,Cleaned_Desc,Description,Day,Month,Year,...,Gap_Days,Rec_30,Rec_60,Rec_90,RecurringStreak_30,RecurringStreak_60,RecurringStreak_90,Is_Recurring_30,Is_Recurring_60,Is_Recurring_90
10,2024-07-01,12500.00,12500.00,<NA>,1,online transfer from 9llabs llc business check...,ONLINE TRANSFER FROM 9LLABS LLC BUSINESS CHECK...,1,July,2024,...,54.0,False,True,False,1,2,0,0,1,0
4,2024-07-08,-9975.00,9975.00,<NA>,0,online transfer to 9llabs llc ref business che...,ONLINE TRANSFER TO 9LLABS LLC REF #IB0NSKMG2J ...,8,July,2024,...,7.0,False,False,False,1,2,0,0,1,0
2,2024-07-24,-3875.00,3875.00,<NA>,0,online transfer to 9llabs llc ref business che...,ONLINE TRANSFER TO 9LLABS LLC REF #IB0NYCGZRL ...,24,July,2024,...,16.0,False,False,False,1,2,0,0,1,0
12,2024-06-21,250.00,250.00,<NA>,1,wells fargo rewards,WELLS FARGO REWARDS,21,June,2024,...,60.0,False,True,False,1,2,0,0,1,0
21,2024-06-06,-683.65,683.65,<NA>,0,city of bellevu util bill sv,CITY OF BELLEVU UTIL BILL 240606 18688005 SV,6,June,2024,...,63.0,False,True,False,0,2,0,0,1,0


In [30]:
account_sorted_df.to_csv("../data/processed/account_sorted.csv", index = False, encoding = "utf-8", date_format = "%Y-%m-%d")